__Reads have already been trimmed with Trimmomatic with dDocent and adapter sequencings removed with adapter.filter.sh. Next perform mapping and alignment, following this tutorial https://www.biostars.org/p/41951/__

In [ ]:
%cd ~/ClownfishGWAS/data/APPC_sequencing/DNA/


In [ ]:
#move trimmed reads to separate directory
!mkdir trimmed_reads 
!mv *R1.fq.gz trimmed_reads
!mv *R2.fq.gz trimmed_reads

In [ ]:
%cd ~/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/


In [ ]:
#screen for contamination and filter out tagged reads simultaneously
screen -S fastq_screen
for i in /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/*.fq.gz; do fastq_screen --tag --filter 1 --outdir fastq_screen/ $i ; 
    done

In [ ]:
#use fastqc and multiqc for basic quality control stats
screen -S fastqc
for i in /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/*.fq.gz; do fastqc -t 60 *.fq.gz  $i ; 
    done

In [ ]:
#visualize sequencing quality for all samples in trimmed_reads directory
multiqc .

In [ ]:
#make bwa index for reference fasta to use in mapping
#!ln -s /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/Genome/reference.fasta reference.fasta
#!mkdir mapping
#!cd ~/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping
!bwa index /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/Genome/reference.fasta

In [ ]:
#map using default parameters but add -M for picard compatibility
!cd ~/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping

In [ ]:
#map to reference genome
#http://seqanswers.com/forums/showthread.php?t=66292
screen -S mapping
    for i in /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/*R1.tagged_filter.fastq.gz;
    do bwa mem -t 50 -M /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/reference.fasta $i > ${i%R1.tagged_filter.fastq.gz}read1.sai; done

In [ ]:
for i in /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/*R2.tagged_filter.fastq.gz;
    do bwa mem -M /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/reference.fasta $i > ${i%R2.tagged_filter.fastq.gz}read2.sai; done

In [ ]:
#copy the filtered and mapped files to mapping directory
cd ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/fastq_screen/
cp *.sai ~/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping/


In [1]:
# I think I should have mapped read one and two at the same time because I'm using bwa mem, redo 11/22/2021
screen -S mapping
for i in /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/fastq_screen/*.R1.tagged_filter.fastq.gz;
    do bwa mem -t 60 -M -R "@RG\tID:$i\tSM:$i\tPL:Illumina" /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/reference.fasta $i.R1.tagged_filter.fastq.gz $i.R2.tagged_filter.fastq.gz | samtools sort -@60 -o $i-RG.bam; done   

In [ ]:
# I think I should have mapped read one and two at the same time because I'm using bwa mem, redo 11/22/2021
#try with filenames... this is what ended up working 11/22/2021
for i in $(cat filenames);
    do bwa mem -t 60 -M -R "@RG\tID:$i\tSM:$i\tPL:Illumina" /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/reference.fasta $i.R1.fq.gz $i.R2.fq.gz  2> bwa.$i.log | samtools sort -@$NUMProc -o $i-RG.bam 2>$i.bam.log; done  

In [ ]:


 2> bwa.$i.log | samtools sort -@$NUMProc -o $i-RG.bam 2>$i.bam.log

In [ ]:
#join read 1 and read 2 files for each sample
#make a list of file names 
!awk '{ print $1 }' /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/popmap > /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/filenames

#join pairs, convert from sam to bam files in screen -S samsort
screen -S samsort
for i in $(cat filenames);
do bwa sampe /local/home/katrinac/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/reference.fasta $i\.read1.sai $i\.read2.sai $i\.R1.tagged_filter.fastq.gz $i\.R2.tagged_filter.fastq.gz | samtools view -bS - >  ~/ClownfishGWAS/data/APPC_sequencing/DNA/trimmed_reads/mapping/$i\.bam; done



In [ ]:
#remove duplicate reads with picard
java -Xms4g -jar /local/home/katrinac/miniconda3/envs/commandline/bin/picard.jar MarkDuplicates I={}-R2.tagged_filter.fastq.gz O={}-RGmd.bam M={}_dup_metrics.txt OPTICAL_DUPLICATE_PIXEL_DISTANCE=2500 TAGGING_POLICY=OpticalOnly &> md.{}.lo      g